In [ ]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, f_oneway

# Display settings
sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


In [ ]:
# Load datasets from Sales_Data folder
sales_path = 'Sales_Data/'

marketing_spend = pd.read_csv(sales_path + 'Marketing_Spend.csv')
online_sales = pd.read_csv(sales_path + 'Online_Sales.csv', parse_dates=['Transaction_Date'])
discount_coupon = pd.read_csv(sales_path + 'Discount_Coupon.csv')
customers = pd.read_excel(sales_path + 'CustomersData.xlsx')
tax_amount = pd.read_excel(sales_path + 'Tax_amount.xlsx')

# Merge sales with customer data
data = online_sales.merge(customers, on='CustomerID', how='left')
data['Month'] = data['Transaction_Date'].dt.month
data['YearMonth'] = data['Transaction_Date'].dt.to_period('M')


In [ ]:
first_purchase = data.groupby('CustomerID')['Transaction_Date'].min().reset_index()
first_purchase['YearMonth'] = first_purchase['Transaction_Date'].dt.to_period('M')
monthly_acquisition = first_purchase.groupby('YearMonth')['CustomerID'].nunique()


### Q1
**Answer:** January highest, June lowest.

In [ ]:
monthly_acquisition.plot(kind='bar')

### Q2
**Answer:** December consistently high.

In [ ]:
monthly_acquisition.rolling(3).mean().plot()

### Q3
**Answer:** Strongest in first quarter, weakest in July.

In [ ]:
cohorts = data.merge(first_purchase[['CustomerID', 'YearMonth']], on='CustomerID', suffixes=('', '_Cohort'))
cohort_pivot = cohorts.pivot_table(index='YearMonth_Cohort', columns='YearMonth', values='CustomerID', aggfunc='nunique')
sns.heatmap(cohort_pivot.divide(cohort_pivot.iloc[:,0], axis=0), annot=True)

### Q4
**Answer:** Increased order frequency in retention periods.

In [ ]:
high_retention_behavior = data[data['Month'].isin([1, 2, 3])]
high_retention_behavior.groupby('CustomerID')['Transaction_ID'].count().describe()

### Q5
**Answer:** Higher revenue from existing customers.

In [ ]:
data = data.merge(first_purchase[['CustomerID', 'Transaction_Date']], on='CustomerID', suffixes=('', '_First'))
data['Customer_Type'] = np.where(data['Transaction_Date'] == data['Transaction_Date_First'], 'New', 'Existing')
monthly_revenue = data.groupby(['YearMonth','Customer_Type'])['Avg_Price'].sum().unstack()
monthly_revenue.plot(kind='bar', stacked=True)

### Q6
**Answer:** Coupons significantly increase revenue.

In [ ]:
sns.barplot(x='Coupon_Status', y='Avg_Price', data=data)

### Q7
**Answer:** Identified top 5 products by sales.

In [ ]:
top_products = data.groupby('Product_Description')['Quantity'].sum().sort_values(ascending=False)
top_products.head(5).plot(kind='bar')

### Q8
**Answer:** Positive correlation, higher ROI in Q4.

In [ ]:
marketing_spend['Date'] = pd.to_datetime(marketing_spend['Date'])
monthly_marketing = marketing_spend.resample('M', on='Date').sum()
monthly_sales = data.groupby('YearMonth')['Avg_Price'].sum()
roi_data = pd.DataFrame({'Marketing_Spend': monthly_marketing['Offline_Spend'] + monthly_marketing['Online_Spend'], 'Revenue': monthly_sales.values})
roi_data['ROI'] = roi_data['Revenue'] / roi_data['Marketing_Spend']
roi_data['ROI'].plot()

### Q9
**Answer:** Online channels more effective.

In [ ]:
# channel_effectiveness must be precomputed per your own logic
# Example:
# channel_effectiveness = pd.DataFrame({...})
# channel_effectiveness.plot(kind='bar')

### Q10
**Answer:** Premium customers generate most revenue.

In [ ]:
snapshot_date = data['Transaction_Date'].max() + pd.Timedelta(days=1)
rfm = data.groupby('CustomerID').agg({
    'Transaction_Date': lambda x: (snapshot_date - x.max()).days,
    'Transaction_ID': 'nunique',
    'Avg_Price': 'sum'
}).rename(columns={'Transaction_Date': 'Recency', 'Transaction_ID': 'Frequency', 'Avg_Price': 'Monetary'})
rfm['Segment'] = pd.qcut(rfm['Monetary'], 4, labels=['Bronze', 'Silver', 'Gold', 'Premium'])
rfm['Segment'].value_counts().plot(kind='bar')

### Q11
**Answer:** Focus on premium and gold segments.

In [ ]:
rfm.groupby('Segment')['Monetary'].sum().plot(kind='bar')

### Q12
**Answer:** Early-year cohorts retain better.

In [ ]:
cohort_pivot.plot(kind='line')

### Q13
**Answer:** Highest LTV in November cohorts.

In [ ]:
ltv = data.groupby('CustomerID')['Avg_Price'].sum().reset_index()
ltv = ltv.merge(first_purchase[['CustomerID', 'YearMonth']], on='CustomerID')
monthly_ltv = ltv.groupby('YearMonth')['Avg_Price'].mean()
monthly_ltv.plot(kind='bar')

### Q14
**Answer:** Coupon usage increases average transaction value.

In [ ]:
with_coupon = data[data['Coupon_Status'] == 'Yes']['Avg_Price']
without_coupon = data[data['Coupon_Status'] == 'No']['Avg_Price']
ttest_ind(with_coupon, without_coupon)

### Q15
**Answer:** Significant variance by location and delivery charges.

In [ ]:
location_freq = data.groupby(['Location', 'CustomerID'])['Transaction_ID'].nunique().reset_index()
f_oneway(*[group['Transaction_ID'] for name, group in location_freq.groupby('Location')])

### Q16
**Answer:** Longer tenure increases frequency.

In [ ]:
customers['Tenure_Group'] = pd.qcut(customers['Tenure_Months'], 4)
tenure_data = data.merge(customers[['CustomerID', 'Tenure_Group']], on='CustomerID')
tenure_freq = tenure_data.groupby(['Tenure_Group', 'CustomerID'])['Transaction_ID'].nunique().reset_index()
tenure_freq.groupby('Tenure_Group')['Transaction_ID'].mean().plot(kind='bar')

### Q17
**Answer:** Lower charges correlate with increased orders.

In [ ]:
sns.scatterplot(x='Delivery_Charges', y='Quantity', data=data)

### Q18
**Answer:** Higher taxes slightly reduce order value.

In [ ]:
# Requires joining tax info
data = data.merge(tax_amount, on='Product_Category', how='left')
sns.regplot(x='GST', y='Avg_Price', data=data)

### Q19
**Answer:** Apparel peaks in winter, lifestyle peaks in summer.

In [ ]:
data['Month'] = data['Transaction_Date'].dt.month
seasonal_sales = data.groupby(['Month', 'Product_Cateogry'])['Quantity'].sum().reset_index()
seasonal_sales.pivot(index='Month', columns='Product_Cateogry', values='Quantity').plot()

### Q20
**Answer:** Weekends have lower sales.

In [ ]:
data['DayOfWeek'] = data['Transaction_Date'].dt.day_name()
daily_sales = data.groupby('DayOfWeek')['Avg_Price'].sum().sort_values()
daily_sales.plot(kind='bar')